# 0. 개발 환경 세팅
- 필요한 패키지 설치
- 모듈 임포트
- CUDA GPU 환경 확인
- 드라이브 마운트

In [ ]:
!pip install -U sentence-transformers
!pip install faiss-cpu --upgrade
!pip install faiss-gpu --upgrade
!pip install text_generation
!pip install gradio
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextIteratorStreamer, TextStreamer
import torch
import gradio as gr

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available")

CUDA is available!


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# 1. 데이터 준비
- id, title, date, running-time, genre, synopsis, tag로 구성되어 있음
- 길이 112를 가지고 있음

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/movie-data.csv")

In [ ]:
df.head() # 상위 5개 데이터 확인

,id,title,date,running-time,genre,synopsis,tag
0,1022789,인사이드 아웃 2,2024년 06월 12일,1시간 37분,"{'애니메이션', '가족', '모험', '코미디'}",13살이 된 라일리의 행복을 위해 매일 바쁘게 머릿속 감정 컨트롤 본부를 운영하는 ...,"{'슬픔', '속편', '애니메이션', '사춘기', '소녀', '공포', '분노',..."
1,519182,슈퍼배드 4,2024년 07월 24일,1시간 34분,"{'애니메이션', '가족', '코미디', '액션'}",슈트-업 하고 악당 전담 처리반 AVL이 된 에이전트 미니언즈와 미니언즈 만큼 귀여...,"{'악당', '속편', '히어로', '환상', '어린이'}"
2,748783,가필드 더 무비,2024년 05월 15일,1시간 41분,"{'애니메이션', '코미디', '가족', '모험'}","세상귀찮 집냥이, 바쁘고 험난한 세상에 던져졌다! 집사 ‘존’과 반려견 ‘오디’를 ...","{'고양이', '빌런', '우정', '라자냐', '코믹', '애니메이션', '만화'..."
3,974262,디센던츠: 레드의 반항,2024년 07월 12일,1시간 32분,"{'판타지', '모험', '가족', '코미디'}",하트 여왕이 오라돈을 차지하려고 하자 그의 반항아 딸 레드는 신데렐라의 완벽주의자 ...,"{'후손', '뮤지컬', '시간여행', '외전', '신데렐라', '원더랜드'}"
4,786892,퓨리오사: 매드맥스 사가,2024년 05월 22일,2시간 29분,"{'액션', '모험', 'SF'}","문명 붕괴 45년 후, 황폐해진 세상 속 누구에게도 알려지지 않은 풍요가 가득한 녹...","{'종말 이후', '전편', '외전', '포학', '남성적', '모녀관계', '아동..."


In [ ]:
len(df)

112

# 2. Retriever 모델 세팅
- text embedding: 허깅페이스 models에 공개된 jhgan/ko-sroberta-multitask 사용 (한국어 문장 임베딩을 위한 BERT 기반 모델)
- 이후 FAISS를 활용하여 사용자의 인풋 쿼리와 데이터가 가장 잘 매치하는 상위 5개 영화를 반환하게 됨

In [ ]:
setence_transformer_name = "jhgan/ko-sroberta-multitask"

In [ ]:
retriever = SentenceTransformer('jhgan/ko-sroberta-multitask', device='cpu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 3. Output 생성 모델 세팅
- 허깅페이스 models에 공개된 kfkas/Llama-2-ko-7b-Chat 사용
- LlaMA2를 기반으로 하여 한국어 데이터 학습
- Text Generation에 특화되어 있음

In [ ]:
model_name = "kfkas/Llama-2-ko-7b-Chat"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map={'':0})

model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# 4. Retriever 모델에 사용되는 코드

- get_embeddings(texts) 함수
    - 한국어 입력 텍스트를 텐서 임베딩으로 반환
    - retriever: 앞서 정의한 jhgan/ko-sroberta-multitask를 사용

- 데이터(df)로부터 tag_embeddings 추출
    - tag_embeddings: df의 태그와 장르 정보를 임베딩 벡터로 생성
    - faiss 라이브러리를 사용하여 최근접 이웃 검색을 준비함
        - faiss.IndexFlatIP: 내적 기반의 평평한 인덱스 생성
        - tag_embeddings.size(-1): 임베딩 벡터의 차원 반환 (인덱스 초기화 시 필요)
        - tag_embeddings.numpy(): PyTorch 텐서를 NumPy 배열로 변환
        - index.add: 변환된 임베딩 벡터를 faiss 인덱스에 추가

- get_movies(query) 함수
    - 사용자 인풋 쿼리에 맞추어 유사한 상위 5개의 영화를 반환
    - query_embedding: 쿼리 문자열을 임베딩 벡터로 생성
    - index.search: query_embedding과 유사한 상위 5개(k=5)의 임베딩 벡터를 faiss 인덱스에서 검색
        - D: 검색된 임베딩 벡터들과 query_embedding 간의 유사도 점수(거리)를 나타내는 배열
        - I: 검색된 임베딩 벡터들의 인덱스를 나타내는 배열
    - df.iloc[I[0]]: 데이터(df)에서 상위 5개 인덱스를 통해 영화 정보를 추출
        - top_5_movies에 담음
    - top_5_movies를 돌며 title과 synopsis를 top_5_results에 담음
    - 최종적으로 영화의 제목과 시놉시스 정보를 담고 있는 top_5_results 배열을 반환

In [ ]:
def get_embeddings(texts):
    return retriever.encode(texts, convert_to_tensor=True)

In [ ]:
tag_embeddings = get_embeddings(df['tag'] + df['genre'])

index = faiss.IndexFlatIP(tag_embeddings.size(-1))
index.add(tag_embeddings.numpy())

In [ ]:
def get_movies(query):
  query_embedding = get_embeddings([query])

  D, I = index.search(query_embedding.numpy(), k=5)

  top_5_movies = df.iloc[I[0]]

  top_5_results = []

  for i, row in top_5_movies.iterrows():
      title = row['title']
      synopsis = row['synopsis']
      top_5_results.append(f"{title} - {synopsis}")

  return top_5_results

In [ ]:
# get_movies 함수 작동 확인

recommend1 = get_movies("심심해. 보면 마음이 두근거리고 도파민 팡팡 터지는 흥미진진한 영화 있어?")
for i in range(len(recommend1)):
  print(recommend1[i])

print("------------------------------------------------------------------------------")

recommend2 = get_movies("등골이 오싹할 만큼 무섭고 소름 돋는 영화 추천해줘")
for i in range(len(recommend2)):
  print(recommend2[i])

스턴트맨 - 잠수 이별을 택하고 후회뿐인 날을 보내던 스턴트맨 ‘콜트’(라이언 고슬링) 영화감독이 된 전 여친 ‘조디’(에밀리 블런트)의 촬영장에 복귀하며 아련한 재회를 기대했지만… 갑자기 주연배우가 사라지고 콜트는 우선 그부터 찾아야만 한다! <데드풀 2> 감독이 말아주는 코미디, 액션, 로맨스 장르 풀코스 액션도 터지고, 사랑도 터지고, 웃음도 터진다!
인사이드 아웃 2 - 13살이 된 라일리의 행복을 위해 매일 바쁘게 머릿속 감정 컨트롤 본부를 운영하는 ‘기쁨’, ‘슬픔’, ‘버럭’, ‘까칠’, ‘소심’. 그러던 어느 날, 낯선 감정인 ‘불안’, ‘당황’, ‘따분’, ‘부럽’이가 본부에 등장하고, 언제나 최악의 상황을 대비하며 제멋대로인 ‘불안’이와 기존 감정들은 계속 충돌한다. 결국 새로운 감정들에 의해 본부에서 쫓겨나게 된 기존 감정들은 다시 본부로 돌아가기 위해 위험천만한 모험을 시작하는데…
이프: 상상의 친구 - 상상의 친구 ‘이프’를 볼 수 있는 능력을 지닌 한 소녀가 아이들에게 잊혀졌던 ‘이프’를 다시 되찾아주기 위해 마법 같은 모험을 시작하는 이야기를 그린 영화
디센던츠 2 - 오란돈 왕국의 공주가 될 예정이었던 '맬'은 숨막히는 생활에 지쳐가고 중압감을 이기지 못한 채 악당들의 섬으로 되돌아가고, 친구들은 그녀를 데려오기 위해 섬으로 뒤쫓아가는데...
데드풀과 울버린 - 히어로 생활에서 은퇴한 후, 평범한 중고차 딜러로 살아가던 ‘데드풀’이 예상치 못한 거대한 위기를 맞아 모든 면에서 상극인 ‘울버린’을 찾아가게 되며 펼쳐지는 도파민 폭발 액션 블록버스터
------------------------------------------------------------------------------
베놈 2: 렛 데어 비 카니지 - ‘베놈'과 완벽한 파트너가 된 ‘에디 브록' 앞에 ‘클리터스 캐서디'가 ‘카니지'로 등장, 앞으로 닥칠 대혼돈의 세상을 예고한다. 대혼돈의 시대가 시작되고, 악을 악으로 처단할 것인가?
분노의 질주: 라이드 오어 다이

# 5. Output 생성 모델에 사용되는 코드

- format_chat_prompt 함수
    - 프롬프트 정의
    - message: 사용자의 인풋
    - chat_history: 이전 대화 내역
    - movies: 추천 영화 목록
    - movies가 존재하면 사용자에게 공감을 해주고 영화를 추천하는 지시문과 예시 대화를 포함한 프롬프트를 정의함
    - movies가 혹여 존재하지 않을 경우에 대해서도 기본 프롬프트를 정의함
    - chat_history의 각 대화 턴을 반복하여 프롬프트에 추가함
    - 최종 프롬프트 반환

- respond 함수
    - message: 사용자의 인풋
    - chat_history: 이전 대화 내역
    - get_movies 함수로부터 추천 영화 목록을 가져옴
    - format_chat_prompt 함수를 사용하여 프롬프트를 생성
    - chat_history에 현재 사용자의 메시지와 빈 응답을 추가
    - 프롬프트 토큰화 후, CUDA로 사용
    - TextIteratorStreamer를 초기화하여 텍스트 생성을 스트리밍
    - 앞서 정의한 모델을 사용하여 텍스트를 생성
    - 생성된 텍스트를 스트리머로부터 받아 model_output에 추가
    - 상위 5개 중 다른 영화들도 함께 추천해줌
    - model_output에서 생성된 텍스트를 하나씩 돌며 전체 문장이 점진적으로 형성되도록 함. 즉, 사용자에게 점진적으로 출력되도록 하여 사용자가 응답을 기다리는 동안 생성된 텍스트를 점차적으로 확인할 수 있도록 설정함

In [ ]:
def format_chat_prompt(message, chat_history, movies):
    if movies:
        prompt = "지시문:\n사용자의 말에 공감을 한 뒤, 사용자에게 영화를 추천하세요.\n"
        message = f"{message}\n영화: {movies[0]}"
    else:
        prompt = "지시문:\n사용자의 요청에 맞는 답변을 생성하세요. 친절하고 부드러운 말투를 사용하세요.\n"

    for turn in chat_history:
        user_message, bot_message = turn
        prompt = f"{prompt}\n사용자: {user_message}\n답변: {bot_message}\n"

    prompt = f"{prompt}\n사용자: {message}"

    return prompt

In [ ]:
def respond(message, chat_history):
    movies = get_movies(message)
    prompt = format_chat_prompt(message, chat_history, movies=movies)
    chat_history = chat_history + [[message, ""]]

    encoded_input = tokenizer(prompt, return_tensors='pt')
    input_ids=encoded_input['input_ids'].cuda()

    streamer = TextIteratorStreamer(tokenizer)

    generated_text = model.generate(input_ids,
                                    streamer=streamer,
                                    pad_token_id=tokenizer.eos_token_id,
                                    do_sample=True,
                                    max_new_tokens=64,
                                    repetition_penalty=1.1,
                                    top_k=20,
                                    top_p=0.92,
                                    )

    model_output = ""
    for new_text in streamer:
        model_output += new_text

    if movies:
      model_output += "\n\n그밖에도"
      for movie in movies[1:]:
        model_output += "\n\n" + movie

      model_output += "\n\n등의 영화가 있습니다."

    acc_text = ""
    for idx, response in enumerate(model_output):
        text_token = response

        if idx == 0:
            text_token = text_token[1:]

        acc_text += text_token
        last_turn = list(chat_history.pop(-1))
        last_turn[-1] += acc_text
        chat_history = chat_history + [last_turn]
        yield "", chat_history
        acc_text = ""

# 6. 사용자 인터페이스 생성

- gradio 사용
    - 메시지 입력 박스
    - 메시지 제출 버튼
    - 콘솔 클리어 버튼

In [ ]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=800)
    msg = gr.Textbox(label="User Input")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])

gr.close_all()
demo.queue().launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://696125e58ecae00fe6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
